# Korelace časových řad
Tento notebook popisuje vývoj volby metod pro zjišťování korelace časových řad.

## Načtení dat
Pro první sadu výpočtů použijeme data z World Bank.

In [ ]:
import pandas as pd
import world_bank_data as wb
import datetime

dataset_props = {
    'SP.DYN.TFRT.IN': {
        'id': 'tfr',
        'name': 'Total fertility rate',
        'description': 'Počet dětí, které by žena mohla mít, kdyby po celý její život platily hodnoty plodnosti podle věku pro daný rok.',
        'unit': 'počet dětí'
    },
    'EN.ATM.CO2E.KT': {
        'id': 'co2_emissions',
        'name': 'Emise CO2',
        'description': 'Emise CO2 vzniklé spalování fosilních paliv a výrobou cementu.',
        'unit': 'kt'
    },
    'NY.GDP.PCAP.PP.CD': {
        'id': 'gdp',
        'name': 'HDP per capita',
        'description': 'HDP per capita v mezinárodních dolarech podle parity kupní síly',
        'unit': 'mezinárodní dolary'
    },
    'SL.TLF.CACT.FE.ZS': {
        'id': 'lfp_female',
        'name': 'Participace žen 15+ v pracovním procesu',
        'description': 'Podíl ekonomicky aktivních žen ve věku 15 a více let (odhad ILOSTAT)',
        'unit': '%'
    },
    'SH.STA.SUIC.P5': {
        'id': 'suicide_mortality_rate',
        'name': 'Sebevražednost',
        'description': 'Hrubý počet sebevražd na 100000 obyvatel (WHO)',
        'unit': 'počet sebevražd'
    }
}

regions = [
    'WLD', # World
    'EUU', # European Union
    'CZE',
    'DEU',
    'GRC',
    'ITA',
    'POL',
    'GBR'
]

datasets = {}
for dataset_id in dataset_props:
    # Collect data since 1980 until now
    year = datetime.date.today().strftime("%Y")
    series = wb.get_series(dataset_id, date='1980:%s' % year, id_or_value='id', simplify_index=True)

    # Process dataset for each selected region
    per_region = {}

    for region in regions:
        data = series[region]

        # Strip NaNs
        index = data.index
        start = 0
        while start <= len(index) - 1:
            if not pd.isnull(data[index[start]]):
                break
            start = start + 1
        end = len(index) - 1
        while end >= 0:
            if not pd.isnull(data[index[end]]):
                break
            end = end - 1
        data = data.iloc[start:(end + 1)] # Last index is exclusive

        # Compute intermediary missing values using interpolation
        data = data.interpolate()

        if data.size < 3:
            print('Skipping %s for %s' % (dataset_id, region))

        per_region[region] = {
            'data': data
        }

    datasets[dataset_id] = per_region

tfr = datasets.pop('SP.DYN.TFRT.IN')

## Statistiky

## Základní vlastnosti

In [ ]:
for dataset_id in datasets:
    for region in datasets[dataset_id]:
        data = datasets[dataset_id][region]['data']
        datasets[dataset_id][region]['properties'] = data.describe()

## Pohled na vývoj TFR
Celosvětový vývoj (WLD) je narozdíl od ostatních grafů škálovaný jinak.

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
fig.set_figwidth(24)
fig.set_figheight(8)
fig.suptitle('TFR in selected regions', fontsize=20)

def plot(i, ylim):
    plt.subplot(2, 4, i + 1)
    plt.plot(tfr[regions[i]]['data'])
    plt.xticks(tfr[regions[i]]['data'].index[0::5])
    plt.title(regions[i])
    plt.ylim(1.0, ylim)

plot(0, 4.0) # Plot WLD separately with a differently scaled y-axis

for i in range(1, len(regions)):
    plot(i, 2.5)

## Augmented Dickey Fuller Test
Otestujeme stacionaritu dat.

In [ ]:
from statsmodels.tsa.stattools import adfuller

def process_adfuller(region, alpha=0.05):
    result = adfuller(region['data'], autolag='AIC')
    region['stationarity'] = {
        'adfuller': {
            'stationary': True if result[1] > alpha else False,
            'statistic': result[0],
            'pvalue': result[1],
            'alpha': alpha,
            'lag': result[2],
            'nobs': result[3],
            'autolag': 'AIC'
        }
    }

print('TFR')
for region in tfr:
    # result = adfuller(tfr[region]['data'], autolag='AIC')
    # print('\n--------------  %s  ------------' % region)
    # print('Result: %s (%s)' % (
    #     'stationary' if result[1] > 0.05 else 'non-stationary',
    #     result[1]
    # ))
    # print(f'ADF Statistic: {result[0]}')
    # print(f'n_lags: {result[2]}')
    # print(f'p-value: {result[1]}')
    # for key, value in result[4].items():
    #     print('Critial Values:')
    #     print(f'   {key}, {value}')
    process_adfuller(tfr[region])
    print('  ' + region + ': ' + ('stationary' if tfr[region]['stationarity']['adfuller']['stationary'] else 'non-stationary'))

for dataset_id in datasets:
    print(dataset_id)
    for region_id in datasets[dataset_id]:
        process_adfuller(datasets[dataset_id][region_id])
        print('  ' + region_id + ': ' + ('stationary' if datasets[dataset_id][region_id]['stationarity']['adfuller']['stationary'] else 'non-stationary'))


## Korelace dvou stacionárních časových řad
- Pearsonův korelační koeficient
- Lineární regrese s t-testem, jehož nulová hypotéza je, že sklon regresní přímky je nulový.
- Vyzkoušení této techniky na několika relativních zpožděních obou časových řad.

In [ ]:
from sklearn.linear_model import LinearRegression # For plotting
from scipy import stats # For the correlation and slope test
import numpy as np

def lag_interval(lagging: tuple, static: tuple, lag: int): 
    """Compute bounds when given intervals are lagged
    
    Parameters:
    - lagging: interval to be lagged
    - static: interval to relate the lag to
    - lag: relative lag, positive value points to the past
    
    Returns:
    - new_lagging: new bounds of the lagged interval or None if invalid
    - new_static: new bounds of the lagged interval or None if invalid
    - length: length of the both intervals of None if interval(s) invalid
    """

    # Compute start points
    lagging_start = lagging[0]
    static_start = lagging_start + lag

    if static_start < static[0]:
        lagging_start += static[0] - static_start
        static_start = static[0]
    
    # Compute end points
    static_end = static[1]
    lagging_end = static_end - lag

    if lagging_end > lagging[1]:
        static_end -= lagging_end - lagging[1]
        lagging_end = lagging[1]
    
    # Validate results
    if static_start > static_end or lagging_start > lagging_end:
        return None, None, None
    
    new_lagging = (lagging_start, lagging_end)
    new_static = (static_start, static_end)
    length = lagging_end - lagging_start

    return new_lagging, new_static, length

def plot_linear_regression(dataset_id, region_id, X, y, lag):
    model = LinearRegression()
    X_2d_array = np.asarray(X).reshape(-1, 1)
    model.fit(X_2d_array, y)
    pred = model.predict(X_2d_array)

    fig = plt.figure()
    fig.set_figwidth(12)
    fig.set_figheight(4)
    fig.suptitle(dataset_id + ' in ' + region_id + ' with lag ' + str(lag))
    plt.subplot(1, 2, 1)
    plt.plot(X.values)
    ax2 = plt.twinx()
    ax2.plot(y.values, color='orange')
    plt.autoscale()
    plt.subplot(1, 2, 2)
    plt.plot(X, pred, color='r')
    plt.scatter(X, y)
    plt.show()

def linear_regression(dataset_id, region_id, X, y, output=False):
    slope, intercept, r_value, p_value, std_err = stats.linregress(X, y)

    # if output:
    #     print("Slope:    ", slope)
    #     print("Intercept:", intercept)
    #     print("R-value:  ", r_value)
    #     print("p-value:  ",  p_value)
    #     print("Std. err: ",  std_err)

    return {
        'correlation': True if p_value <= 0.05 and abs(r_value) >= 0.4 else False,
        'linear_regression': {
            'slope': slope,
            'intercept': intercept,
            'p_value': p_value,
            'std_err': std_err
        },
        'pearson_product_moment_correlation': {
            'r_value': r_value
        }
    }

def linear_regression_lagged(dataset_id, region_id, X, y, maxlags=5, output='none'):
    """maxlags: lag series relatively between -maxlags and maxlags"""
    
    min_data_lenght = 5

    X_interval = (int(X.index[0]), int(X.index[-1]))
    X_length = X_interval[1] - X_interval[0]

    y_interval = (int(y.index[0]),
        int(y.index[-1]))
    y_length = y_interval[1] - y_interval[0]

    min_length = int(max(min_data_lenght, min(X_length, y_length) / 2))

    relations = {}

    for lag in range(-maxlags, maxlags):
        X_lagged, y_lagged, length = lag_interval(X_interval, y_interval, lag)

        if length != None and length >= min_length:
            # print(dataset_id + ' ' + region_id + ' ' + str(lag))

            primary = X
            other = y

            primary.index = primary.index.astype(int)
            other.index = other.index.astype(int)
            primary = primary.loc[(primary.index >= X_lagged[0]) & (primary.index <= X_lagged[1])]
            other = other.loc[(other.index >= y_lagged[0]) & (other.index <= y_lagged[1])]

            relations[lag] = linear_regression(dataset_id, region_id, primary, other, output=output)
    
    max_r_value = 0
    max_r_value_lag = None
    for lag in relations:
        cur_r_value = abs(relations[lag]['pearson_product_moment_correlation']['r_value'])
        if cur_r_value > max_r_value:
            max_r_value = cur_r_value
            max_r_value_lag = lag
    datasets[dataset_id][region_id]['relation'] = relations[max_r_value_lag]
    datasets[dataset_id][region_id]['relation']['lag'] = max_r_value_lag
    
    if output == 'always' or (output == 'corr' and relations[max_r_value_lag]['correlation']):
        X_lagged, y_lagged, length = lag_interval(X_interval, y_interval, max_r_value_lag)
        primary = X
        other = y
        primary.index = primary.index.astype(int)
        other.index = other.index.astype(int)
        primary = primary.loc[(primary.index >= X_lagged[0]) & (primary.index <= X_lagged[1])]
        other = other.loc[(other.index >= y_lagged[0]) & (other.index <= y_lagged[1])]
        plot_linear_regression(dataset_id, region_id, primary, other, max_r_value_lag)
        print(datasets[dataset_id][region_id]['relation'])
        # print(('stationary' if datasets[dataset_id][region_id]['stationarity']['adfuller']['stationary'] else 'non-stationary'))

# # Process all stationary pairs per each region
# for region_id in regions:
#     for dataset_id in datasets:
#         if tfr[region_id]['stationarity']['adfuller']['stationary'] \
#             and datasets[dataset_id][region_id]['stationarity']['adfuller']['stationary']:
#             linear_regression_lagged(dataset_id, region_id)
#             print(dataset_id + ' in ' + region_id + ': '
#                 + ('YES' if datasets[dataset_id][region_id]['relation']['correlation'] else 'no'))
#             # if datasets[dataset_id][region_id]['relation']['correlation']:
#             #     plot_linear_regression(dataset_id, region_id)
#             #     print(datasets[dataset_id][region_id]['relation'])

linear_regression_lagged('EN.ATM.CO2E.KT', 'CZE', tfr['CZE']['data'], datasets['EN.ATM.CO2E.KT']['CZE']['data'], 10, output='always')
linear_regression_lagged('NY.GDP.PCAP.PP.CD', 'GRC', tfr['GRC']['data'], datasets['NY.GDP.PCAP.PP.CD']['GRC']['data'], 10, output='always')

### Korelace po diferenciaci
Korelační koeficient (r-value) se ve většině případů zvyšuje spolu s lagem. Může to být způsobené tím, že analyzujeme stacionaritu celé časové řady a ne až vyříznuté části používanou pro výpočet korelace. Proto je pravděpodobně aplikovaný způsob výpočtu nevhodný. Zkusíme časovou řadu diferencovat až poté ji analyzovat.

In [ ]:
def differenced_linear_regression(dataset_id, region_id):
    join = pd.merge(
        tfr[region_id]['data'].diff().iloc[1:],
        datasets[dataset_id][region_id]['data'].diff().iloc[1:],
        on='Year',
        how='inner')
    X = join['SP.DYN.TFRT.IN']
    y = join[dataset_id]

    result = linear_regression(
        dataset_id,
        region_id,
        X,
        y)
    datasets[dataset_id][region_id]['relation'] = result
    plot_linear_regression(dataset_id, region_id, X, y, 0)
    print(result)

differenced_linear_regression('EN.ATM.CO2E.KT', 'CZE')
differenced_linear_regression('NY.GDP.PCAP.PP.CD', 'GRC')

Tyto výsledky vypadají slibněji, analyzujeme celý stacionární dataset a prohlédneme si podrobně ty, u kterých bude nalezena korelace.

In [ ]:
for region_id in regions:
    # Diff all series
    tfr[region_id]['differenced'] = {
            1: tfr[region_id]['data'].diff().iloc[1:]
        }
    for dataset_id in datasets:
        datasets[dataset_id][region_id]['differenced'] = {
            1: datasets[dataset_id][region_id]['data'].diff().iloc[1:]
        }
        # Try linear regression with lagging
        if datasets[dataset_id][region_id]['stationarity']['adfuller']['stationary']:
            linear_regression_lagged(
                dataset_id, region_id,
                tfr[region_id]['differenced'][1],
                datasets[dataset_id][region_id]['differenced'][1],
                10, output='corr')

### Normalizace
Výsledky vypadají ve většině případů smysluplně.
Sklon regresní přímky je ale závislý na rozdílech mezi řády hodnot jednotlivých časových řad a moc proto nevypovídá pro porovnání napříč regresemi.
Data proto po diferenciaci ještě normalizujeme na rozsah od -1 do 1.

In [ ]:
for region_id in regions:
    # Normalize all series
    diffed = tfr[region_id]['differenced'][1]
    tfr[region_id]['differenced_normalized'] = {
            1: (diffed - diffed.mean()) / (diffed.max() - diffed.min())
        }
    for dataset_id in datasets:
        diffed = datasets[dataset_id][region_id]['differenced'][1]
        datasets[dataset_id][region_id]['differenced_normalized'] = {
            1: (diffed - diffed.mean()) / (diffed.max() - diffed.min())
        }
        # Try linear regression with lagging
        linear_regression_lagged(
            dataset_id, region_id,
            tfr[region_id]['differenced_normalized'][1],
            datasets[dataset_id][region_id]['differenced_normalized'][1],
            10, output='corr')

## Korelace nestacionárních časových řad
Stejný postup vyzkoušíme i na nestacionárních časových řadách.
Diferenciace by měla nestacionaritu vyřešit, abychom opět porovnávali změny v trendu a ne pouze jeho celkové směřování. 
Pro testování získáme více nestacionárních dat.

### Načtení dat

In [ ]:
# %%capture

# API to fetch the datasets from
api = 'https://ec.europa.eu/eurostat/api/dissemination/sdmx/2.1/data/%s?format=TSV'

# Regions to collect the datasets for
# Eurostat country codes along with codes used here
eurostat_regions = {
    'CZ': 'CZE',
    'DE': 'DEU',
    'EL': 'GRC',
    'IT': 'ITA',
    'PL': 'POL',
    'UK': 'GBR'
}

# Datasets to collect along with their metadata
dataset_props = {
    'DEMO_NSINAGEC': [
        {
            'id': 'first_marriages_women',
            'name': 'První sňatky žen',
            'description': 'Počet poprvé se vdávajících žen',
            'unit': 'počet sňatků',
            'filter': {
                'sex': 'F',
                'age': 'TOTAL'
            },
        },
        {
            'id': 'first_marriages_men',
            'name': 'První sňatky mužů',
            'description': 'Počet poprvé se ženících mužů',
            'unit': 'počet sňatků',
            'filter': {
                'sex': 'M',
                'age': 'TOTAL'
            },
        },
    ],
    'EDUC_UOE_PERD03': [
        {
            'id': 'teachers_women_ed1',
            'name': 'Podíl žen mezi vyučujícími prvního stupně',
            'description': 'Procento zastoupení žen mezi vyučujícími prvního stupně',
            'unit': '%',
            'filter': {
                'isced11': 'ED1'
            }
        },
        {
            'id': 'teachers_women_ed3',
            'name': 'Podíl žen mezi vyučujícími středního vzdělání',
            'description': 'Procento zastoupení žen mezi vyučujícími středního vzdělání',
            'unit': '%',
            'filter': {
                'isced11': 'ED3'
            }
        }
    ],
    'EDUC_UOE_FINE06': [
        {
            'id': 'education_expenditure',
            'name': 'Státní výdaje na vzdělávání',
            'description': 'Výdaje na vzdelávání od předškolní do vysokoškolské úrovně',
            'unit': '% HDP',
            'filter': {
                'isced11': 'ED02-8'
            }
        }
    ],
    'TPS00106': [
        {
            'id': 'social_benefits_health',
            'name': 'Výdaje na zdravotní péči',
            'description': 'Podíl státních výdajů na zdravotní péči v porovnání se všemi sociálními výdaji',
            'unit': '%',
            'filter': {
                'spdeps': 'SICK'
            }
        },
        {
            'id': 'social_benefits_old_age',
            'name': 'Výdaje na starobní důchody',
            'description': 'Podíl státních výdajů na starobní důchody v porovnání se všemi sociálními výdaji',
            'unit': '%',
            'filter': {
                'spdeps': 'OLD'
            }
        },
        {
            'id': 'social_benefits_family',
            'name': 'Výdaje na podporu rodin',
            'description': 'Podíl státních výdajů na podporu rodin v porovnání se všemi sociálními výdaji',
            'unit': '%',
            'filter': {
                'spdeps': 'FAM'
            }
        },
        {
            'id': 'social_benefits_unemployment',
            'name': 'Výdaje na podporu v nezaměstnanosti',
            'description': 'Podíl státních výdajů na podporu v nezaměstnanosti v porovnání se všemi sociálními výdaji',
            'unit': '%',
            'filter': {
                'spdeps': 'UNEMPLOY'
            }
        }
    ],
    'YTH_DEMO_030': [
        {
            'id': 'age_when_leaving_parents_men',
            'name': 'Průměrný věk osamostatnění potomků - mužů',
            'description': 'Odhadovaný průměrný věk, kdy mladí dospělí opouštějí rodičovskou domácnost',
            'unit': 'věk',
            'filter': {
                'sex': 'M',
            }
        },
        {
            'id': 'age_when_leaving_parents_women',
            'name': 'Průměrný věk osamostatnění potomků - žen',
            'description': 'Odhadovaný průměrný věk, kdy mladí dospělí opouštějí rodičovskou domácnost',
            'unit': 'věk',
            'filter': {
                'sex': 'F',
            }
        }
    ],
    'CRIM_PRIS_AGE': [
        {
            'id': 'prisoners',
            'name': 'Počet vězňů',
            'description': 'Počet vězňů na 100 000 obyvatel',
            'unit': 'počet vězňů',
            'filter': {
                'age': 'TOTAL',
                'sex': 'T',
                'unit': 'P_HTHAB'
            }
        }
    ]
}

def strip_nans(data):
    """Remove leading and trailing rows with missing values"""

    index = data.index

    start = 0
    while start <= len(index) - 1:
        if not pd.isnull(data[index[start]]):
            break
        start = start + 1

    end = len(index) - 1
    while end >= 0:
        if not pd.isnull(data[index[end]]):
            break
        end = end - 1
    
    return data.iloc[start:(end + 1)] # Last index is exclusive

for eurostat_id in dataset_props:
    data = pd.read_csv(api % eurostat_id, sep='\t')
    datasets[eurostat_id] = {}

    # Prepare data for filtering

    # Split leading column into separate columns
    leading_col = data.columns[0]
    leading_cols = data.columns[0].split(sep='\\')[0].split(sep=',')
    data[leading_cols] = pd.DataFrame(data[leading_col].apply(lambda x: x.split(sep=',')).to_list(), index=data.index)
    data.drop(leading_col, axis=1, inplace=True)

    # Replace ':' followed by optional flags with NaN
    data.replace(regex=':.*', value=np.NaN, inplace=True)

    # Strip trailing whitespace from column names
    data.rename(mapper=lambda x: x.strip(), axis=1, inplace=True)

    # print(data.head(10))

    # Filter out datasets
    for dataset in dataset_props[eurostat_id]:
        filtered_data = pd.DataFrame(data)
        for filter_by in dataset['filter']:
            filtered_data = filtered_data[data[filter_by] == dataset['filter'][filter_by]]
        
        # Extract per-country data
        for region in eurostat_regions:
            region_data = filtered_data[data['geo'] == region]

            if region_data.size == 0:
                continue
            
            # Create Series without the values used for filtering
            region_data = region_data.transpose()
            region_data.drop(index=leading_cols, inplace=True)
            region_data = region_data[region_data.columns[0]].squeeze()

            # Strip flags and trailing spaces from the values
            region_data = region_data.apply(lambda x: str(x).split(sep=' ')[0]).astype(np.float64)

            # Strip leading and trailing NaNs and interpolate intermediary missing values
            region_data = strip_nans(region_data)
            region_data = region_data.interpolate()

            region_data.index.rename('Year', inplace=True)
            region_data.name = eurostat_id

            if region_data.size > 2:
                datasets[eurostat_id][eurostat_regions[region]] = {
                    'data': region_data
                }

### Test stacionarity

In [ ]:
for dataset_id in dataset_props:
    print(dataset_id)
    for region_id in datasets[dataset_id]:
        process_adfuller(datasets[dataset_id][region_id])
        print('  ' + region_id + ': ' + ('stationary' if datasets[dataset_id][region_id]['stationarity']['adfuller']['stationary'] else 'non-stationary'))

### Korelace po diferenciaci a normalizaci pro nestacionární časové řady
Ověříme výsledky stejného postupu jako u stacionárních řad.

In [ ]:
for dataset_id in datasets:
    for region_id in datasets[dataset_id]:
        diffed = datasets[dataset_id][region_id]['data'].diff().iloc[1:]
        datasets[dataset_id][region_id]['differenced_normalized'] = {
            1: (diffed - diffed.mean()) / (diffed.max() - diffed.min())
        }
        if not datasets[dataset_id][region_id]['stationarity']['adfuller']['stationary']:
            # Try linear regression with lagging
            linear_regression_lagged(
                dataset_id, region_id,
                tfr[region_id]['differenced_normalized'][1],
                datasets[dataset_id][region_id]['differenced_normalized'][1],
                10, output='corr')

Výsledky ověříme ještě pohledem na původní nediferencovaná data.

In [ ]:
for dataset_id in datasets:
    for region_id in datasets[dataset_id]:
        if not datasets[dataset_id][region_id]['stationarity']['adfuller']['stationary'] \
            and datasets[dataset_id][region_id]['relation']['correlation']:

            
            primary = tfr[region_id]['data']
            primary.index = primary.index.astype(int)
            other = datasets[dataset_id][region_id]['data']
            other.index = other.index.astype(int)

            join = pd.merge(
                primary,
                other,
                on='Year',
                how='inner')

            X = join['SP.DYN.TFRT.IN']
            y = join[dataset_id]

            plot_linear_regression(dataset_id, region_id, X, y, 0)

Je patrné, že změny v trendu se u všech dvojic skutečně podobají (případně je třeba brát v úvahu zpoždění, při kterém byla korelace nalezena).